# 汇聚层 Pooling

希望逐渐降低隐藏表示的空间分辨率、汇聚信息，这样随着我们在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大

具有双重目的：
- 降低卷积层对位置的敏感性
- 降低对空间降采样表示的敏感性

## 1. maximum pooling 和 average pooling

In [37]:
'''pooling函数'''
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i+p_h, j: j+p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i+p_h, j: j+p_w].mean()
    return Y

In [38]:
'''验证 max'''
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [39]:
pool2d(X, (2, 2), mode='avg')

tensor([[2., 3.],
        [5., 6.]])

## 2. Padding and Stirde
汇聚层也可以更改填充和步幅

In [40]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X.shape

torch.Size([1, 1, 4, 4])

In [41]:
'''默认情况下，深度学习框架中的步幅与汇聚窗口的大小相同。 因此，如果我们使用形状为(3, 3)的汇聚窗口，那么默认情况下，我们得到的步幅形状为(3, 3)。'''
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [42]:
'''填充步幅可以手动设定'''
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [43]:
'''填充步幅可以手动设定'''
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

## 3. 多通道

汇聚层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总。 这意味着汇聚层的输出通道数与输入通道数相同

In [44]:
'''将在通道维度上连结张量X和X + 1，以构建具有2个通道的输入'''
X = torch.cat((X, X + 1), 1)
print(X)
print(X.shape) # 在第2维进行拼接

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])
torch.Size([1, 2, 4, 4])


In [45]:
'''汇聚后输出通道的数量仍然是2'''
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])